In [1]:
from policyengine_us import Simulation
from policyengine_core.reforms import Reform
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from policyengine_core.charts import format_fig

/Users/daphnehansell/miniconda3/envs/policyengine/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
reform = Reform.from_dict(
    {
        "gov.contrib.aca.ptc_additional_bracket.in_effect": {
            "2026-01-01.2100-12-31": True
        }
    },
    country_id="us",
)

reform2 = Reform.from_dict(
    {
        "gov.contrib.aca.ptc_simplified_bracket.in_effect": {
            "2026-01-01.2100-12-31": True
        }
    },
    country_id="us",
)

reform3 = Reform.from_dict({
  "gov.aca.ptc_phase_out_rate[0].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[1].amount": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[2].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[3].amount": {
    "2026-01-01.2100-12-31": 0.02
  },
  "gov.aca.ptc_phase_out_rate[4].amount": {
    "2026-01-01.2100-12-31": 0.04
  },
  "gov.aca.ptc_phase_out_rate[5].amount": {
    "2026-01-01.2100-12-31": 0.06
  },
  "gov.aca.ptc_phase_out_rate[6].amount": {
    "2026-01-01.2100-12-31": 0.085
  },
  "gov.aca.ptc_income_eligibility[2].amount": {
    "2026-01-01.2100-12-31": True
  }
}, country_id="us")


In [3]:
situation_wisconsin = {
  "people": {
    "parent": {
      "age": {
        "2026": 27
      }
    },
    "child": {
      "age": {
        "2026": 3
      }
    }
  },
  "families": {
    "your family": {
      "members": [
        "parent",
        "child"
      ]
    }
  },
  "spm_units": {
    "your household": {
      "members": [
        "parent",
        "child"
      ]
    }
  },
  "tax_units": {
    "your tax unit": {
      "members": [
        "parent",
        "child"
      ]
    }
  },
  "households": {
    "your household": {
      "members": [
        "parent",
        "child"
      ],
      "state_name": {
        "2026": "WI"
      },
      "county_fips": {
        "2026": "55079"
      }
    }
  },
  "marital_units": {
    "parent marital unit": {
      "members": [
        "parent"
      ]
    }
  },
  "axes": [
    [
      {
        "name": "employment_income",
        "count": 800,
        "min": 0,
        "max": 120000
      }
    ]
  ]
}

In [4]:
simulation_wisconsin = Simulation(
    situation=situation_wisconsin,
)
reformed_simulation_wisconsin = Simulation(
    situation=situation_wisconsin,
    reform=reform,
)
reformed_simulation_wisconsin2 = Simulation(
    situation=situation_wisconsin,
    reform=reform2,
)
reformed_simulation_wisconsin3 = Simulation(
    situation=situation_wisconsin,
    reform=reform3,
)

In [5]:
import copy
import pandas as pd
from policyengine_us import Simulation

PERIOD = 2026

# ------------------------------------------------------------------
# Helper: get the tax unit's 2026 FPG from the situation
# ------------------------------------------------------------------
def get_tax_unit_fpg(base_situation: dict, period=PERIOD) -> float:
    """Return the tax unit FPG for the given situation/year (first tax unit)."""
    sit = copy.deepcopy(base_situation)
    sit.pop("axes", None)

    # Ensure income isn't interfering (FPG shouldn't depend on income, but be safe)
    for person in sit["people"].values():
        person.setdefault("employment_income", {})
        person["employment_income"][str(period)] = 0

    sim = Simulation(situation=sit)
    fpg = sim.calculate("tax_unit_fpg", map_to="tax_unit", period=period)[0]
    return float(fpg)

# ------------------------------------------------------------------
# 1) Convenience: run a one-income Wisconsin single parent simulation
# ------------------------------------------------------------------
def calc_ptc_for_income(base_situation: dict, income: float, *, reform_to_use=None, period=PERIOD):
    """
    Return ACA PTC (household-level) for the given income and year.
    """
    sit = copy.deepcopy(base_situation)
    sit.pop("axes", None)

    # Assign all income to the parent
    sit["people"]["parent"]["employment_income"] = {str(period): income}

    sim = Simulation(situation=sit, reform=reform_to_use)
    return sim.calculate("aca_ptc", map_to="household", period=period)[0]

# ------------------------------------------------------------------
# 2) Build targets from model-derived FPG and compute PTCs
# ------------------------------------------------------------------
fpg_2026 = get_tax_unit_fpg(situation_wisconsin, period=PERIOD)

percent_targets = {
    "138 % FPL": 1.38,
    "300 % FPL": 3.00,
    "400 % FPL": 4.00,
}

rows = []
for label, mult in percent_targets.items():
    inc = round(fpg_2026 * mult, 2)
    rows.append({
        "income_label": f"{label} (${inc:,.2f})",
        "income_usd": inc,
        "ptc_baseline":   calc_ptc_for_income(situation_wisconsin, inc, reform_to_use=None, period=PERIOD),
        "ptc_stepped_proposal": calc_ptc_for_income(situation_wisconsin, inc, reform_to_use=reform, period=PERIOD),
        "ptc_linear_proposal": calc_ptc_for_income(situation_wisconsin, inc, reform_to_use=reform2, period=PERIOD),
        "ptc_ira_extension": calc_ptc_for_income(situation_wisconsin, inc, reform_to_use=reform3, period=PERIOD),
    })

ptc_df = pd.DataFrame(rows)
ptc_df

,income_label,income_usd,ptc_baseline,ptc_stepped_proposal,ptc_linear_proposal,ptc_ira_extension
0,"138 % FPL ($29,897.10)",29897.10,3623.720703,4710.216309,4219.903809,4710.216309
1,"300 % FPL ($64,993.70)",64993.70,0.000000,628.612061,0.000000,696.855469
2,"400 % FPL ($86,658.27)",86658.27,0.000000,0.000000,0.000000,782.626953


In [6]:
# Get household-level values for Wisconsin
household_income_wisconsin = simulation_wisconsin.calculate("employment_income", map_to="household", period=2026)
baseline_wisconsin_per_capita_chip = simulation_wisconsin.calculate("per_capita_chip", map_to="household", period=2026)
baseline_wisconsin_aca_ptc = simulation_wisconsin.calculate("aca_ptc", map_to="household", period=2026)
baseline_wisconsin_medicaid_cost = simulation_wisconsin.calculate("medicaid_cost", map_to="household", period=2026)
baseline_wisconsin_net_income_including_health_benefits = simulation_wisconsin.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)
baseline_wisconsin_slcsp = simulation_wisconsin.calculate("slcsp", map_to="household", period=2026)

reform_wisconsin_per_capita_chip = reformed_simulation_wisconsin.calculate("per_capita_chip", map_to="household", period=2026)
reform_wisconsin_aca_ptc = reformed_simulation_wisconsin.calculate("aca_ptc", map_to="household", period=2026)
reform_wisconsin_medicaid_cost = reformed_simulation_wisconsin.calculate("medicaid_cost", map_to="household", period=2026)
reform_wisconsin_net_income_including_health_benefits = reformed_simulation_wisconsin.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)

reform2_wisconsin_per_capita_chip = reformed_simulation_wisconsin2.calculate("per_capita_chip", map_to="household", period=2026)
reform2_wisconsin_aca_ptc = reformed_simulation_wisconsin2.calculate("aca_ptc", map_to="household", period=2026)
reform2_wisconsin_medicaid_cost = reformed_simulation_wisconsin2.calculate("medicaid_cost", map_to="household", period=2026)
reform2_wisconsin_net_income_including_health_benefits = reformed_simulation_wisconsin2.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)

reform3_wisconsin_per_capita_chip = reformed_simulation_wisconsin3.calculate("per_capita_chip", map_to="household", period=2026)
reform3_wisconsin_aca_ptc = reformed_simulation_wisconsin3.calculate("aca_ptc", map_to="household", period=2026)
reform3_wisconsin_medicaid_cost = reformed_simulation_wisconsin3.calculate("medicaid_cost", map_to="household", period=2026)
reform3_wisconsin_net_income_including_health_benefits = reformed_simulation_wisconsin3.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)

# Calculate total benefits for each scenario
baseline_wisconsin_total = [sum(x) for x in zip(baseline_wisconsin_per_capita_chip, baseline_wisconsin_aca_ptc, baseline_wisconsin_medicaid_cost)]
reform_wisconsin_total = [sum(x) for x in zip(reform_wisconsin_per_capita_chip, reform_wisconsin_aca_ptc, reform_wisconsin_medicaid_cost)]
reform2_wisconsin_total = [sum(x) for x in zip(reform2_wisconsin_per_capita_chip, reform2_wisconsin_aca_ptc, reform2_wisconsin_medicaid_cost)]
reform3_wisconsin_total = [sum(x) for x in zip(reform3_wisconsin_per_capita_chip, reform3_wisconsin_aca_ptc, reform3_wisconsin_medicaid_cost)]

In [7]:
GRAY = "#808080"
BLUE_PRIMARY = "#2C6496"
TEAL_ACCENT = "#39C6C0"
DARK_GRAY = "#616161"
PURPLE = "#9467BD"


In [8]:
# Create Wisconsin graph
fig_wisconsin = go.Figure()

# Add baseline traces (solid lines)
fig_wisconsin.add_trace(go.Scatter(
    x=household_income_wisconsin, 
    y=baseline_wisconsin_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Baseline)', 
    line=dict(color=GRAY, width=2)
))

# Add reform traces
fig_wisconsin.add_trace(go.Scatter(
    x=household_income_wisconsin, 
    y=reform_wisconsin_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Stepped Proposal)', 
    line=dict(color=BLUE_PRIMARY, width=2)
))

fig_wisconsin.add_trace(go.Scatter(
    x=household_income_wisconsin, 
    y=reform2_wisconsin_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Linear Proposal)', 
    line=dict(color=TEAL_ACCENT, width=2)
))

fig_wisconsin.add_trace(go.Scatter(
    x=household_income_wisconsin, 
    y=reform3_wisconsin_aca_ptc, 
    mode='lines', 
    name='ACA PTC (IRA Extension)', 
    line=dict(color=PURPLE, width=2)
))


# Update layout
fig_wisconsin.update_layout(
    title='Wisconsin Household (Single parent, age 27, with child, age 3) - Program Benefits by Income Level',
    xaxis_title='Household Income',
    yaxis_title='Benefit Amount',
    legend_title='Programs',
    xaxis=dict(tickformat='$,.0f', range=[0, 120000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)

fig_wisconsin = format_fig(fig_wisconsin)
fig_wisconsin.show()

In [9]:
#Household net income graphs
import plotly.graph_objects as go

# ---------- Wisconsin single parent ----------
fig_wi = go.Figure()

# Baseline (solid)
fig_wi.add_trace(go.Scatter(
    x=household_income_wisconsin,
    y=baseline_wisconsin_net_income_including_health_benefits,
    mode='lines',
    name='Baseline',
    line=dict(color=GRAY, width=2)
))

# Reform 1 (stepped)
fig_wi.add_trace(go.Scatter(
    x=household_income_wisconsin,
    y=reform_wisconsin_net_income_including_health_benefits,
    mode='lines',
    name='Stepped Proposal',
    line=dict(color=BLUE_PRIMARY, width=2)
))

# Reform 2 (linear)
fig_wi.add_trace(go.Scatter(
    x=household_income_wisconsin,
    y=reform2_wisconsin_net_income_including_health_benefits,
    mode='lines',
    name='Linear Proposal',
    line=dict(color=TEAL_ACCENT, width=2)
))

# Reform 3 (IRA extension)
fig_wi.add_trace(go.Scatter(
    x=household_income_wisconsin,
    y=reform3_wisconsin_net_income_including_health_benefits,
    mode='lines',
    name='IRA Extension',
    line=dict(color=PURPLE, width=2)
))

# Layout
fig_wi.update_layout(
    title='Wisconsin Household (Single parent, age 27, with child, age 3) – Health-Adjusted Net Income by Household Income',
    xaxis_title='Household Income',
    yaxis_title='Health-Adjusted Net Income',
    legend_title='Scenario',
    xaxis=dict(tickformat='$,.0f', range=[0, 120_000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)

fig_wi = format_fig(fig_wi)
fig_wi.show()

# --- Δ Health-adjusted net income (Reform – Baseline) ---
delta_wi = (
    reform_wisconsin_net_income_including_health_benefits
    - baseline_wisconsin_net_income_including_health_benefits
)

delta_wi2 = (
    reform2_wisconsin_net_income_including_health_benefits
    - baseline_wisconsin_net_income_including_health_benefits
)

delta_wi3 = (
    reform3_wisconsin_net_income_including_health_benefits
    - baseline_wisconsin_net_income_including_health_benefits
)

fig_delta_wi = go.Figure()

fig_delta_wi.add_trace(go.Scatter(
    x=household_income_wisconsin,
    y=delta_wi,
    mode='lines',
    name='Stepped Proposal',
    line=dict(color=BLUE_PRIMARY, width=2)
))

fig_delta_wi.add_trace(go.Scatter(
    x=household_income_wisconsin,
    y=delta_wi2,
    mode='lines',
    name='Linear Proposal',
    line=dict(color=TEAL_ACCENT, width=2)
))

fig_delta_wi.add_trace(go.Scatter(
    x=household_income_wisconsin,
    y=delta_wi3,
    mode='lines',
    name='IRA Extension',
    line=dict(color=PURPLE, width=2)
))

fig_delta_wi.update_layout(
    title='Wisconsin Household (Single parent, age 27, with child, age 3) – Impact of Premium Tax Credit Proposals',
    xaxis_title='Household Income',
    yaxis_title='Δ Net Income',
    xaxis=dict(tickformat='$,.0f', range=[0, 120_000]),
    yaxis=dict(tickformat='$,.0f', zeroline=True, zerolinewidth=1),
    height=600,
    width=1000,
    legend=dict(orientation='h')
)

fig_delta_wi = format_fig(fig_delta_wi)
fig_delta_wi.show()

In [10]:
# ========================================================================
# MTR CALCULATION FOR WISCONSIN HOUSEHOLD
# Uses axes-based simulation for net income, then calculates MTR manually
# ========================================================================

# Step 1: Create situation with axes for vectorized net income calculation
situation_wi_for_mtr = {
    "people": {
        "parent": {
            "age": {"2026": 27},
            "employment_income": {"2026": 0},
            "self_employment_income": {"2026": 0}
        },
        "child": {
            "age": {"2026": 3},
            "employment_income": {"2026": 0},
            "self_employment_income": {"2026": 0}
        }
    },
    "families": {
        "your family": {
            "members": ["parent", "child"]
        }
    },
    "spm_units": {
        "your household": {
            "members": ["parent", "child"]
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": ["parent", "child"]
        }
    },
    "households": {
        "your household": {
            "members": ["parent", "child"],
            "state_name": {"2026": "WI"},
            "county_fips": {"2026": "55079"}
            
        }
    },
    "marital_units": {
        "parent marital unit": {
            "members": ["parent"]
        }
    },
    "axes": [[
        {
            "name": "employment_income",
            "min": 0,
            "max": 120_000,
            "count": 200,
            "period": "2026"
        }
    ]]
}

# Step 2: Calculate baseline net income using axes
sim_wi_baseline = Simulation(situation=situation_wi_for_mtr)
household_income_wi_mtr = sim_wi_baseline.calculate("employment_income", map_to="household", period=2026)
baseline_wi_net_income = sim_wi_baseline.calculate(
    "household_net_income_including_health_benefits",
    map_to="household",
    period=2026
)

# Step 3: Calculate reform net income using axes
sim_wi_reform = Simulation(situation=situation_wi_for_mtr, reform=reform)
reform_wi_net_income = sim_wi_reform.calculate(
    "household_net_income_including_health_benefits",
    map_to="household",
    period=2026
)

# Step 3b: Calculate reform2 net income using axes
sim_wi_reform2 = Simulation(situation=situation_wi_for_mtr, reform=reform2)
reform2_wi_net_income = sim_wi_reform2.calculate(
    "household_net_income_including_health_benefits",
    map_to="household",
    period=2026
)

# Step 3c: Calculate reform3 net income using axes
sim_wi_reform3 = Simulation(situation=situation_wi_for_mtr, reform=reform3)
reform3_wi_net_income = sim_wi_reform3.calculate(
    "household_net_income_including_health_benefits",
    map_to="household",
    period=2026
)

# Step 4: Calculate MTR from adjacent points
def calc_mtr(incomes, net_incomes):
    """Calculate MTR between adjacent income points."""
    mtrs = []
    mtr_incomes = []
    for i in range(len(incomes) - 1):
        income_change = incomes[i + 1] - incomes[i]
        net_change = net_incomes[i + 1] - net_incomes[i]
        if income_change > 0 and not np.isnan(net_incomes[i]) and not np.isnan(net_incomes[i + 1]):
            mtr = 1 - (net_change / income_change)
            mtrs.append(mtr)
            mtr_incomes.append((incomes[i] + incomes[i + 1]) / 2)
    return np.array(mtr_incomes), np.array(mtrs)

baseline_wi_mtr_incomes, baseline_wi_mtrs = calc_mtr(household_income_wi_mtr, baseline_wi_net_income)
reform_wi_mtr_incomes, reform_wi_mtrs = calc_mtr(household_income_wi_mtr, reform_wi_net_income)
reform2_wi_mtr_incomes, reform2_wi_mtrs = calc_mtr(household_income_wi_mtr, reform2_wi_net_income)
reform3_wi_mtr_incomes, reform3_wi_mtrs = calc_mtr(household_income_wi_mtr, reform3_wi_net_income)

# Step 5: Create the chart
fig_wisconsin_mtr = go.Figure()

fig_wisconsin_mtr.add_trace(go.Scatter(
    x=baseline_wi_mtr_incomes,
    y=np.clip(baseline_wi_mtrs, -1.0, 1.0),
    mode='lines',
    name='Baseline',
    line=dict(color=GRAY, width=2),
    hovertemplate='Income: $%{x:,.0f}<br>Baseline MTR: %{y:.1%}<extra></extra>'
))

fig_wisconsin_mtr.add_trace(go.Scatter(
    x=reform_wi_mtr_incomes,
    y=np.clip(reform_wi_mtrs, -1.0, 1.0),
    mode='lines',
    name='Stepped Proposal',
    line=dict(color=BLUE_PRIMARY, width=2),
    hovertemplate='Income: $%{x:,.0f}<br>Stepped MTR: %{y:.1%}<extra></extra>'
))

fig_wisconsin_mtr.add_trace(go.Scatter(
    x=reform2_wi_mtr_incomes,
    y=np.clip(reform2_wi_mtrs, -1.0, 1.0),
    mode='lines',
    name='Linear Proposal',
    line=dict(color=TEAL_ACCENT, width=2),
    hovertemplate='Income: $%{x:,.0f}<br>Linear MTR: %{y:.1%}<extra></extra>'
))

fig_wisconsin_mtr.add_trace(go.Scatter(
    x=reform3_wi_mtr_incomes,
    y=np.clip(reform3_wi_mtrs, -1.0, 1.0),
    mode='lines',
    name='IRA Extension',
    line=dict(color=PURPLE, width=2),
    hovertemplate='Income: $%{x:,.0f}<br>IRA MTR: %{y:.1%}<extra></extra>'
))

fig_wisconsin_mtr.update_layout(
    title='Marginal tax rate including health benefits - Wisconsin single parent (age 27) with child (age 3)',
    xaxis_title='Employment income',
    yaxis_title='Marginal tax rate',
    xaxis=dict(
        tickformat='$,.0f',
        range=[0, 120_000],
        gridcolor='lightgray',
        showgrid=True
    ),
    yaxis=dict(
        tickformat='.0%',
        range=[-1.0, 1.0],
        gridcolor='lightgray',
        showgrid=True,
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='gray'
    ),
    height=600,
    width=1000,
    hovermode='x unified',
    plot_bgcolor='white',
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig_wisconsin_mtr = format_fig(fig_wisconsin_mtr)
fig_wisconsin_mtr.show()